In [1]:
import pandas as pd
import os
import duckdb

In [9]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/workspaces/spotify_insights/airflow/credentials/google_credentials.json'
#source_path = "gs://spotify-insights-pipeline-data/raw-tables"
source_path = "gs://spotify-insights-pipeline-data/stg_spotify_streams/dtype_dedup"

In [10]:
stg_spotify_streams = pd.read_parquet(source_path, engine="pyarrow")
stg_spotify_streams = stg_spotify_streams[stg_spotify_streams['username'] == 'joseph.higaki']
stg_spotify_streams

,stream_id,streamed_at,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,...,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,incognito_mode,offline_timestamp,username,streamed_year
58061,3a485a35bc0e69b13dd346231d6e0d4f,2014-12-23 00:37:58+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",300826,US,66.165.162.68,Smells Like Teen Spirit,Nirvana,Nirvana,spotify:track:4hy4fb5D1KL50b3sng9cjw,...,None,appload,trackdone,True,True,True,True,NaN,joseph.higaki,2014
58062,182d23a8a69eedb73e7c3a44a2bab6d5,2014-12-23 00:40:47+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",169133,US,66.165.162.68,Rape Me,Nirvana,Nirvana,spotify:track:47KVHb6cOVBZbmXQweE5p7,...,None,trackdone,trackdone,True,True,True,True,NaN,joseph.higaki,2014
58063,cccab707a4b2d4d2f29f6688d8cc281a,2014-12-23 00:46:44+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",356080,US,66.165.162.68,Sweet Child O' Mine,Guns N' Roses,Appetite For Destruction,spotify:track:7o2CTH4ctstm8TNelqjb51,...,None,trackdone,trackdone,True,True,True,True,NaN,joseph.higaki,2014
58064,98f0e6fbf3499b3c74bee057a80619b4,2014-12-23 00:49:35+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",170800,US,66.165.162.68,Rock And Roll All Nite,KISS,Greatest Kiss,spotify:track:3qei6Em6zqxjFuPfhhghWM,...,None,trackdone,trackdone,True,True,True,True,NaN,joseph.higaki,2014
58065,1a8efb967a73de58831d3a28b8c75b7f,2014-12-23 00:54:26+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",290280,US,66.165.162.68,Message In A Bottle - Remastered 2003,The Police,Reggatta De Blanc,spotify:track:2inge47GpvkHhe0VCXP8Zd,...,None,trackdone,trackdone,True,True,True,True,NaN,joseph.higaki,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152728,4f67f885b94cd9fde7d741614509ccb5,2024-12-04 06:55:39+00:00,android,282653,ES,79.153.169.248,Voodoo,Godsmack,Godsmack,spotify:track:5uGsG0LfotfWDq6hql4h53,...,None,trackdone,trackdone,True,True,True,True,1.733295e+09,joseph.higaki,2024
152729,2d77649d593863f0944b8cbb3370cd12,2024-12-04 06:58:59+00:00,android,199906,ES,79.153.169.248,Last Resort,Papa Roach,Infest,spotify:track:5W8YXBz9MTIDyrpYaCg2Ky,...,None,trackdone,trackdone,True,True,True,True,1.733295e+09,joseph.higaki,2024
152730,3d09fac078d3518bb28d431729652109,2024-12-04 07:04:05+00:00,android,304840,ES,79.153.169.248,Uprising,Muse,The Resistance,spotify:track:4VqPOruhp5EdPBeR92t6lQ,...,None,trackdone,trackdone,True,True,True,True,1.733296e+09,joseph.higaki,2024
152731,1f6cf1e59bf4e9c4c984cbc5e29240c2,2024-12-04 07:26:58+00:00,android,148181,ES,79.117.89.98,Killing In The Name,Rage Against The Machine,Rage Against The Machine - XX (20th Anniversar...,spotify:track:59WN2psjkt1tyaxjspN8fp,...,None,trackdone,unknown,True,True,True,True,1.733296e+09,joseph.higaki,2024


In [ ]:
,window window_by_streamed_at_stream_id as (PARTITION BY streamed_at, stream_id)


In [19]:
with duckdb.connect() as con:    
    query = """
        with with_count as (
            select             
            count() over (window_by_streamed_at) as count_by_streamed_at,
            lag(conn_country) over (order by streamed_at, stream_id) as lag_conn_country,
            lead(conn_country) over (order by streamed_at, stream_id) as lead_conn_country,
            *
            from stg_spotify_streams
            window window_by_streamed_at as (PARTITION BY streamed_at)
            
        )
        select * 
        from with_count 
        where lag_conn_country <> conn_country
        and conn_country = 'AD'
        order by streamed_at, stream_id
    """        
    df = con.execute(query).fetchdf()
df

,count_by_streamed_at,lag_conn_country,lead_conn_country,stream_id,streamed_at,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,...,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,incognito_mode,offline_timestamp,username,streamed_year
0,13,ES,AD,05356e18570be330fa21764b92285037,2022-01-05 21:41:08+00:00,"Android OS 10 API 29 (HUAWEI, EML-L09)",324466,AD,194.158.73.101,Chlorine,...,None,clickrow,trackdone,True,True,True,True,1.641414e+12,joseph.higaki,2022
1,5,ES,AD,278b3bd1a717195e6fe1a6a9b6e82717,2022-03-18 19:08:04+00:00,"Android OS 11 API 30 (Xiaomi, 21091116AG)",240792,AD,46.172.224.78,All Along the Watchtower,...,None,clickrow,trackdone,True,True,True,True,NaN,joseph.higaki,2022
2,1,ES,AD,35919e8485961613a1e984e8befd6e8c,2022-12-10 09:22:37+00:00,android,105404,AD,109.111.118.89,Thumbing My Way,...,None,trackdone,logout,True,True,True,True,1.670663e+09,joseph.higaki,2022
3,1,ES,AD,e8637fe9bec4b237321874f14b6712d2,2022-12-11 07:44:24+00:00,android,196853,AD,109.111.118.89,L.S.F.,...,None,clickrow,trackdone,True,True,True,True,1.670744e+09,joseph.higaki,2022
4,1,ES,AD,e59dc916eb82983980201b42c1fb6d3c,2022-12-17 20:55:42+00:00,android,405445,AD,85.94.179.36,Dance With The Devil,...,None,trackdone,logout,True,True,True,True,1.671310e+09,joseph.higaki,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,1,NL,NL,43b4662389c55eb63391af9dfef8b974,2024-04-12 19:56:09+00:00,android,46697,AD,46.172.234.224,Wild Thing,...,None,trackdone,logout,True,True,True,True,1.712950e+09,joseph.higaki,2024
73,1,NL,NL,13a9b55fbf080d2afea83d245980cfbb,2024-04-13 08:37:33+00:00,android,121733,AD,85.94.178.15,Let Me Blow Ya Mind,...,None,trackdone,logout,True,True,True,True,1.712995e+09,joseph.higaki,2024
74,1,NL,NL,b05eaed0f34c11f8308e5d49ecf4718b,2024-04-13 18:43:33+00:00,android,74648,AD,46.172.234.224,C'mere,...,None,trackdone,logout,True,True,True,True,1.713033e+09,joseph.higaki,2024
75,1,NL,NL,2dba7027bc3629ebc7f84533d9cb9040,2024-04-14 11:53:38+00:00,android,208968,AD,109.111.99.36,Who Are You,...,None,trackdone,logout,True,True,True,True,1.713095e+09,joseph.higaki,2024


In [18]:
with duckdb.connect() as con:    
    query = """
        with stg as (
            select                         
            *
            from stg_spotify_streams
            window window_by_streamed_at as (PARTITION BY streamed_at)
            
        )
        select * 
        from stg 
        where streamed_at < '2023-12-15'
        order by streamed_at desc, stream_id desc
    """        
    df = con.execute(query).fetchdf()
df

,stream_id,streamed_at,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,...,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,incognito_mode,offline_timestamp,username,streamed_year
0,16a5e60916964ce2c2b8463ab76675e9,2023-12-10 18:32:15+00:00,android,237586,ES,79.116.106.86,Constant Craving - 2010 Remaster,k.d. lang,Recollection,spotify:track:0wCrg1LhgPcGMw51qqpI6k,...,None,trackdone,logout,True,True,True,True,1.702232e+09,joseph.higaki,2023
1,3009a8abb4724843118d8c11baad1482,2023-12-10 18:12:55+00:00,android,189293,ES,79.117.78.199,Right Here Right Now,Jesus Jones,Doubt,spotify:track:3fcGGP62sllcNEhuFJVYeC,...,None,clickrow,trackdone,True,True,True,True,1.702232e+09,joseph.higaki,2023
2,addb39abe221c15b83a27366ad6d1ba0,2023-12-10 18:09:46+00:00,android,289629,ES,79.117.71.232,What's Up?,4 Non Blondes,"Bigger, Better, Faster, More !",spotify:track:0jWgAnTrNZmOGmqgvHhZEm,...,None,clickrow,endplay,True,True,True,True,1.702231e+09,joseph.higaki,2023
3,c67d62a86225ba17a4b0a92091ee2a0a,2023-12-10 18:04:56+00:00,android,217164,ES,79.117.71.232,Because the Night - Live Unplugged,"10,000 Maniacs",MTV Unplugged,spotify:track:5VzvK7YiwWbRePA2JOzhYq,...,None,trackdone,endplay,True,True,True,True,1.702231e+09,joseph.higaki,2023
4,ae328ffc1d8b316ec2b992004539e25f,2023-12-10 18:01:18+00:00,android,208826,ES,79.117.72.201,(I Can't Help) Falling In Love With You,UB40,The Very Best Of,spotify:track:7aR1QwbQa26sjDXiVMLik4,...,None,clickrow,trackdone,True,True,True,True,1.702231e+09,joseph.higaki,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84469,1a8efb967a73de58831d3a28b8c75b7f,2014-12-23 00:54:26+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",290280,US,66.165.162.68,Message In A Bottle - Remastered 2003,The Police,Reggatta De Blanc,spotify:track:2inge47GpvkHhe0VCXP8Zd,...,None,trackdone,trackdone,True,True,True,True,NaN,joseph.higaki,2014
84470,98f0e6fbf3499b3c74bee057a80619b4,2014-12-23 00:49:35+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",170800,US,66.165.162.68,Rock And Roll All Nite,KISS,Greatest Kiss,spotify:track:3qei6Em6zqxjFuPfhhghWM,...,None,trackdone,trackdone,True,True,True,True,NaN,joseph.higaki,2014
84471,cccab707a4b2d4d2f29f6688d8cc281a,2014-12-23 00:46:44+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",356080,US,66.165.162.68,Sweet Child O' Mine,Guns N' Roses,Appetite For Destruction,spotify:track:7o2CTH4ctstm8TNelqjb51,...,None,trackdone,trackdone,True,True,True,True,NaN,joseph.higaki,2014
84472,182d23a8a69eedb73e7c3a44a2bab6d5,2014-12-23 00:40:47+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",169133,US,66.165.162.68,Rape Me,Nirvana,Nirvana,spotify:track:47KVHb6cOVBZbmXQweE5p7,...,None,trackdone,trackdone,True,True,True,True,NaN,joseph.higaki,2014
